In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 w

In [2]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
from sklearn.model_selection import train_test_split
from nltk.translate.bleu_score import sentence_bleu
from bs4 import BeautifulSoup
import os

# Load dataset
dataset = load_dataset("HuggingFaceM4/WebSight", "v0.2", split="train[:1%]")

# Tokenizer and Model Initialization
tokenizer = AutoTokenizer.from_pretrained("t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

# Data Preprocessing
def preprocess_function(examples):
    inputs = examples['image']  # Placeholder for image preprocessing
    targets = tokenizer(examples['html'], truncation=True, padding="max_length", max_length=512)
    return {"input_ids": inputs, "labels": targets["input_ids"]}

# Preprocess dataset
processed_dataset = dataset.map(preprocess_function, batched=True)

# Split dataset into train and eval
train_data, eval_data = train_test_split(processed_dataset, test_size=0.1)

# Training Arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True
)

# Trainer Setup
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data
)

# Model Training
trainer.train()

# Save the trained model
model.save_pretrained("./trained_model")
tokenizer.save_pretrained("./trained_model")

# Evaluation with BLEU Score and Structural Validity
def evaluate_model(trainer, eval_data):
    predictions = trainer.predict(eval_data)
    decoded_preds = tokenizer.batch_decode(predictions.predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(predictions.label_ids, skip_special_tokens=True)

    bleu_scores = [sentence_bleu([label.split()], pred.split()) for label, pred in zip(decoded_labels, decoded_preds)]
    structural_validity = [is_valid_html(pred) for pred in decoded_preds]

    avg_bleu = sum(bleu_scores) / len(bleu_scores)
    valid_html_ratio = sum(structural_validity) / len(structural_validity)

    print(f"Average BLEU Score: {avg_bleu}")
    print(f"Valid HTML Ratio: {valid_html_ratio}")

# Check if generated HTML is structurally valid
def is_valid_html(html):
    try:
        BeautifulSoup(html, "html.parser")
        return True
    except Exception as e:
        return False

# Run Evaluation
evaluate_model(trainer, eval_data)

# Demonstration on Google Colab
def generate_html(image_input):
    inputs = tokenizer(image_input, return_tensors="pt", padding=True)
    outputs = model.generate(**inputs)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example of generating HTML from an image input
example_input = "path_to_example_image.jpg"  # Replace with actual image input
generated_html = generate_html(example_input)
print(generated_html)

# Fine-tuning (Optional)
def fine_tune_model(train_data, eval_data):
    fine_tune_args = Seq2SeqTrainingArguments(
        output_dir="./fine_tuned_results",
        evaluation_strategy="epoch",
        learning_rate=1e-5,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=5,
        predict_with_generate=True,
        fp16=True
    )

    fine_tune_trainer = Seq2SeqTrainer(
        model=model,
        args=fine_tune_args,
        train_dataset=train_data,
        eval_dataset=eval_data
    )

    fine_tune_trainer.train()
    fine_tune_trainer.save_model("./fine_tuned_model")

# Uncomment to fine-tune the model
# fine_tune_model(train_data, eval_data)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.16k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/738 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/738 [00:00<?, ?it/s]

(…)-00000-of-00738-80a58552f2fb3344.parquet:   0%|          | 0.00/418M [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:651: UserWarning: Not enough free disk space to download the file. The expected file size is: 415.28 MB. The target location /root/.cache/huggingface/hub/datasets--HuggingFaceM4--WebSight/blobs only has 396.12 MB free disk space.
  warnings.warn(


(…)-00214-of-00738-7dc0267fbae957a4.parquet:   0%|          | 0.00/415M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:651: UserWarning: Not enough free disk space to download the file. The expected file size is: 413.16 MB. The target location /root/.cache/huggingface/hub/datasets--HuggingFaceM4--WebSight/blobs only has 0.00 MB free disk space.
  warnings.warn(


(…)-00215-of-00738-9c52e50647fcbf94.parquet:   0%|          | 0.00/413M [00:00<?, ?B/s]

OSError: [Errno 28] No space left on device